In [ ]:
import glob
import os
import warnings

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import lib.helpers as helpers

from lib.keras_video import VideoFrameGenerator

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, auc

from tensorflow.keras.layers import GlobalAveragePooling2D, LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.kears.applications.vgg16 import VGG16

from IPython import get_ipython

get_ipython().run_line_magic("matplotlib", "inline")
warnings.filterwarnings("ignore")

In [ ]:
DS = os.path.sep
DATASET_DIR = "dataset"

BATCH_SIZE = 2
NB_COLOR_CHANNELS = 3
NB_FRAMES = 30
RESOLUTION = (224, 224)

In [ ]:
class_names = helpers.get_generated_class_names(DATASET_DIR)
glob_pattern = helpers.get_generated_glob_pattern(DATASET_DIR)

In [ ]:
train_dataset_generator = VideoFrameGenerator(
    batch_size=BATCH_SIZE,
    classes=class_names,
    glob_pattern=glob_pattern,
    nb_channel=NB_COLOR_CHANNELS,
    nb_frames=NB_FRAMES,
    seed=42,
    split_test=.20,
    split_val=.20,
    target_shape=RESOLUTION,
    transformation=None,
    use_frame_cache=False
)

test_dataset_generator = train_dataset_generator.get_test_generator()
valid_dataset_generator = train_dataset_generator.get_validation_generator()

In [ ]:
input_shape = (NB_FRAMES,) + RESOLUTION + (NB_COLOR_CHANNELS,)

In [ ]:
vgg16_model = VGG16(
  include_top=False,
  input_shape=input_shape[1:],
  weights="imagenet"
)

vgg16_model.trainable = False

In [ ]:
model = Sequential()

model.add(TimeDistributed(vgg16_model, input_shape=input_shape))
model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(LSTM(256))
model.add(Dense(1024, activation="relu"))
model.add(Dropout(.2))
model.add(Dense(int(len(class_names)), activation="softmax"))

model.summary()

In [ ]:
model_epochs = 5
model_earlystopping = EarlyStopping(monitor="loss", patience=10)
model_checkpoint = ModelCheckpoint(
  filepath="checkpoint/vgg16-lstm.h5",
  mode="max",
  monitor="val_acc",
  save_best_only=True,
  verbose=1
)

model_callbacks = [model_earlystopping, model_checkpoint]

In [ ]:
model.compile(
  loss="categorical_crossentropy",
  metrics=["acc"],
  optimizer="adam"
)

In [ ]:
model_history = model.fit(
    train_dataset_generator,
    callbacks=model_callbacks,
    epochs=model_epochs,
    validation_data=valid_dataset_generator
)

In [ ]:
model.save("model/dance/vgg16-lstm-5e.h5")

In [ ]:
helpers.get_visualized_graph(
  plots=[model_history.history["acc"], model_history.history["val_acc"]],
  title="Model Accuracy",
  x_label="Epoch",
  y_label="Accuracy",
  legend=["train", "test"]
).show()

In [ ]:
helpers.get_visualized_graph(
  plots=[model_history.history["loss"], model_history.history["val_loss"]],
  title="Model Loss",
  x_label="Epoch",
  y_label="Loss",
  legend=["train", "test"]
).show()

In [ ]:
model_history_dataframe = pd.DataFrame(model_history.history)
model_history_fpath = "history/dance/vgg16-lstm-5e.csv"

with open(model_history_fpath, mode="w") as history_file:
    model_history_dataframe.to_csv(history_file)

In [ ]:
model.evaluate(test_dataset_generator)

In [ ]:
y_prediction_max, y_true = helpers.get_populated_y_data(
    batch_size=BATCH_SIZE,
    generator=test_dataset_generator,
    model=model
)

In [ ]:
score_accuracy, score_precision, score_recall, score_f1 = helpers.get_calculated_score(y_true, y_prediction_max)

print(f"Accuracy Score\t: {np.round(score_accuracy, 3)}")
print(f"Precision Score\t: {np.round(score_precision, 3)}")
print(f"Recall Score\t: {np.round(score_recall, 3)}")
print(f"F1 Score\t: {np.round(score_f1, 3)}")

In [ ]:
test_class_names = test_dataset_generator.classes

print(classification_report(
    y_true,
    y_prediction_max,
    target_names=test_class_names
))

In [ ]:
confusion_matrix_result = confusion_matrix(y_true, y_prediction_max)

sns.heatmap(
    confusion_matrix_result,
    annot=True,
    cmap="Blues"
)

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_prediction_max, pos_label=6)
score_auc = auc(fpr, tpr)

print(f"AUC Score\t: {np.round(score_auc, 3)}")

In [ ]:
plt.plot(fpr, tpr, marker=".")
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

plt.show()

In [ ]:
from tensorflow.keras.models import load_model

checkpoint_model = load_model("checkpoint/vgg16-lstm.h5")
checkpoint_model.evaluate(test_dataset_generator)

In [ ]:
y_prediction_max, y_true = helpers.get_populated_y_data(
    batch_size=BATCH_SIZE,
    generator=test_dataset_generator,
    model=model
)

In [ ]:
score_accuracy, score_precision, score_recall, score_f1 = helpers.get_calculated_score(y_true, y_prediction_max)

print(f"Accuracy Score\t: {np.round(score_accuracy, 3)}")
print(f"Precision Score\t: {np.round(score_precision, 3)}")
print(f"Recall Score\t: {np.round(score_recall, 3)}")
print(f"F1 Score\t: {np.round(score_f1, 3)}")

In [ ]:
test_class_names = test_dataset_generator.classes

print(classification_report(
    y_true,
    y_prediction_max,
    target_names=test_class_names
))

In [ ]:
confusion_matrix_result = confusion_matrix(y_true, y_prediction_max)

sns.heatmap(
    confusion_matrix_result,
    annot=True,
    cmap="Blues"
)

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_prediction_max, pos_label=6)
score_auc = auc(fpr, tpr)

print(f"AUC Score\t: {np.round(score_auc, 3)}")

In [ ]:
plt.plot(fpr, tpr, marker=".")
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

plt.show()